# Workshop PyTorch (Avanzado): Segmentación de Tumores Cerebrales con U-Net

## Objetivo del Notebook
En esta sesión práctica avanzada, construiremos un sistema de segmentación de imágenes médicas de principio a fin. Utilizaremos PyTorch para entrenar una red neuronal U-Net que identifique y delinee tumores en imágenes de resonancia magnética (MRI) cerebrales.

**Aprenderás a:**
- Implementar un pipeline de datos robusto con Dataset y DataLoader.
- Utilizar Transfer Learning para acelerar el entrenamiento y mejorar el rendimiento.
- Aprovechar librerías de alto nivel como segmentation-models-pytorch para no reinventar la rueda.
- Definir métricas de evaluación específicas para segmentación, como el Dice Score.
- Visualizar los resultados de forma intuitiva para interpretar el rendimiento del modelo.

## Prerequisitos
**Conocimientos:**
- Fundamentos de Python y Programación Orientada a Objetos.
- Conceptos básicos de Machine Learning.
- Fundamentos de PyTorch (Tensores, nn.Module, bucle de entrenamiento básico).

**Librerías necesarias:**
- torch y torchvision
- segmentation-models-pytorch
- scikit-learn
- numpy y matplotlib
- Pillow (PIL)
- tqdm (para barras de progreso)


## Paso 1: Imports y Configuración Inicial
Primero, importamos todas las librerías que necesitaremos y configuramos nuestro entorno. Es una buena práctica agrupar todas las importaciones al principio.

In [ ]:
# Celda 1: Imports y configuración
import os
import glob
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# --- PyTorch y ecosistema ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

# La librería mágica para modelos de segmentación
# !pip install segmentation-models-pytorch
import segmentation_models_pytorch as smp

# --- Utilidades ---
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm # Para barras de progreso elegantes

# Configuración de reproducibilidad y dispositivo
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
set_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {DEVICE}")

## Paso 2: Carga y Exploración de Datos

Trabajaremos con el dataset LGG MRI Segmentation de Kaggle. Contiene imágenes de resonancia magnética de cerebros junto con máscaras segmentadas manualmente que indican la ubicación de gliomas de bajo grado (LGG).

**Instrucciones de descarga:**
- Descarga el dataset desde Kaggle.
- Descomprímelo en una carpeta llamada `lgg-mri-segmentation` en el mismo directorio que este notebook.

In [ ]:
# Celda 2: Verificar ruta de datos y explorar archivos
DATA_DIR = './lgg-mri-segmentation/kaggle_3m/'

if not os.path.isdir(DATA_DIR):
    print(f"Error: El directorio '{DATA_DIR}' no fue encontrado.")
    print("Asegúrate de haber descargado y descomprimido los datos en la ruta correcta.")
else:
    print(f"Directorio de datos encontrado en: {DATA_DIR}")
    all_files = glob.glob(os.path.join(DATA_DIR, '*/*.tif'))
    all_images = sorted([p for p in all_files if 'mask' not in p])
    all_masks = sorted([p for p in all_files if 'mask' in p])
    print(f"Total de imágenes: {len(all_images)}")
    print(f"Total de máscaras: {len(all_masks)}")
    if all_images:
        print(f"Ejemplo de ruta de imagen: {all_images[0]}")
    else:
        print("No se encontraron imágenes.")

## Visualización de una imagen y su máscara

Antes de construir el pipeline de datos, visualicemos una muestra del dataset para entender el formato de las imágenes y las máscaras.

In [ ]:
# Celda: Visualizar una imagen y su máscara correspondiente
def show_image_and_mask(img_path: str, mask_path: str):
    """Muestra una imagen MRI y su máscara de segmentación lado a lado."""
    img = Image.open(img_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.imshow(img)
    ax1.set_title("Imagen MRI Original")
    ax1.axis('off')
    ax2.imshow(mask, cmap='gray')
    ax2.set_title("Máscara del Tumor (Ground Truth)")
    ax2.axis('off')
    plt.show()

# Tomamos una muestra aleatoria para visualizar
if 'all_images' in locals() and all_images:
    sample_idx = random.randint(0, len(all_images) - 1)
    sample_image_path = all_images[sample_idx]
    sample_mask_path = sample_image_path.replace('.tif', '_mask.tif')
    print(f"Mostrando muestra #{sample_idx}")
    show_image_and_mask(sample_image_path, sample_mask_path)

## Paso 3: El Pipeline de Datos de PyTorch (Dataset y DataLoader)

**Teoría breve:**
- `Dataset`: Clase abstracta de PyTorch para crear objetos que entienden nuestro conjunto de datos.
- `DataLoader`: Encargado de agrupar los datos en batches, barajarlos y cargarlos en paralelo.

A continuación, definimos las transformaciones necesarias para las imágenes y máscaras.

In [ ]:
# Celda: Definir las transformaciones
IMAGE_SIZE = 256
transforms = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(), # Convierte la imagen a Tensor y normaliza los píxeles a [0, 1]
])

In [ ]:
# Celda: Crear la clase Dataset personalizada
class BrainTumorDataset(Dataset):
    """
    Dataset personalizado para cargar imágenes MRI de tumores cerebrales y sus máscaras.
    """
    def __init__(self, image_paths: list, transform: T.Compose = None):
        self.image_paths = image_paths
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx: int):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        mask_path = img_path.replace('.tif', '_mask.tif')
        mask = Image.open(mask_path).convert("L")
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        mask = (mask > 0.5).float()
        return image, mask

In [ ]:
# Celda: Dividir los datos y crear los DataLoaders
train_paths, val_paths = train_test_split(all_images, test_size=0.2, random_state=SEED)

train_dataset = BrainTumorDataset(train_paths, transform=transforms)
val_dataset = BrainTumorDataset(val_paths, transform=transforms)

BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Tamaño del set de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del set de validación: {len(val_dataset)}")
print(f"N° de lotes de entrenamiento: {len(train_loader)}")
print(f"N° de lotes de validación: {len(val_loader)}")

## Paso 4: El Modelo - U-Net y Transfer Learning

**Teoría breve:**
- **U-Net:** Arquitectura diseñada para segmentación biomédica, con encoder, decoder y skip connections.
- **Transfer Learning:** Usamos un encoder preentrenado (por ejemplo, ResNet34) para aprovechar características aprendidas en ImageNet.
- **Librería smp:** segmentation-models-pytorch nos permite crear U-Net con diferentes encoders fácilmente.

In [ ]:
# Celda: Crear el modelo U-Net con smp
model = smp.Unet(
    encoder_name="resnet34",        # Encoder backbone
    encoder_weights="imagenet",     # Transfer Learning
    in_channels=3,                  # Imágenes RGB
    classes=1,                      # Máscara binaria
).to(DEVICE)

# print(model)  # Descomenta para ver la arquitectura completa

## Paso 5: Función de Pérdida y Optimizador

**Teoría breve:**
- **Dice Loss:** Métrica robusta para segmentación, ideal cuando hay desbalance entre fondo y objeto.
- **Adam:** Optimizador recomendado para comenzar en tareas de segmentación.

In [ ]:
# Celda: Definir la función de pérdida y el optimizador
loss_fn = smp.losses.DiceLoss(mode=smp.losses.BINARY_MODE, from_logits=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

## Paso 6: Entrenamiento del Modelo

**Teoría breve:**
- El bucle de entrenamiento alterna entre fases de entrenamiento y validación.
- Se calcula la pérdida y el Dice Score en cada época para monitorear el aprendizaje.

In [ ]:
# Celda: Bucle de entrenamiento y validación
EPOCHS = 1
history = {'train_loss': [], 'val_loss': [], 'dice_score': []}

for epoch in range(EPOCHS):
    print(f"--- Epoch {epoch + 1}/{EPOCHS} ---")
    # --- Fase de Entrenamiento ---
    model.train()
    epoch_train_loss = 0
    for images, masks in tqdm(train_loader, desc="Entrenamiento"):
        images, masks = images.to(DEVICE), masks.to(DEVICE)
        outputs = model(images)
        loss = loss_fn(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    # --- Fase de Validación ---
    model.eval()
    epoch_val_loss = 0
    epoch_dice_score = 0
    with torch.no_grad():
        for images, masks in tqdm(val_loader, desc="Validación"):
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            outputs = model(images)
            epoch_val_loss += loss_fn(outputs, masks).item()
            preds = (torch.sigmoid(outputs) > 0.5).float()
            tp, fp, fn, tn = smp.metrics.get_stats(preds.long(), masks.long(), mode='binary')
            dice_score = smp.metrics.f1_score(tp, fp, fn, tn, reduction='micro-imagewise')
            epoch_dice_score += dice_score
    avg_train_loss = epoch_train_loss / len(train_loader)
    avg_val_loss = epoch_val_loss / len(val_loader)
    avg_dice_score = epoch_dice_score / len(val_loader)
    history['train_loss'].append(avg_train_loss)
    history['val_loss'].append(avg_val_loss)
    history['dice_score'].append(avg_dice_score)
    print(f"Pérdida Entrenamiento: {avg_train_loss:.4f}")
    print(f"Pérdida Validación:   {avg_val_loss:.4f}")
    print(f"Dice Score Validación: {avg_dice_score:.4f}")

## Paso 7: Análisis de Resultados

Un entrenamiento no está completo si no analizamos su rendimiento. Graficaremos la pérdida y el Dice Score a lo largo de las épocas.

In [ ]:
# Celda: Graficar el historial de entrenamiento
def plot_history(history: dict):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    ax1.plot(history['train_loss'], label='Pérdida Entrenamiento')
    ax1.plot(history['val_loss'], label='Pérdida Validación')
    ax1.set_title('Pérdida a lo largo de las Épocas')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Dice Loss')
    ax1.legend()
    ax1.grid(True)
    ax2.plot(history['dice_score'], label='Dice Score Validación')
    ax2.set_title('Dice Score a lo largo de las Épocas')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Dice Score')
    ax2.legend()
    ax2.grid(True)
    plt.show()

plot_history(history)

## Visualización de Predicciones

Mostramos la imagen original, la máscara real y la máscara predicha por el modelo para varias muestras del conjunto de validación.

In [ ]:
# Celda: Visualizar predicciones del modelo
def visualize_predictions(model: nn.Module, loader: DataLoader, num_samples: int = 5):
    model.eval()
    samples_shown = 0
    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(DEVICE), masks.to(DEVICE)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            images_np = images.cpu().numpy()
            masks_np = masks.cpu().numpy()
            preds_np = preds.cpu().numpy()
            for i in range(images.size(0)):
                if samples_shown >= num_samples:
                    plt.show()
                    return
                fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 5))
                img_display = np.transpose(images_np[i], (1, 2, 0))
                ax1.imshow(img_display)
                ax1.set_title("Input MRI")
                ax1.axis('off')
                ax2.imshow(masks_np[i].squeeze(), cmap='gray')
                ax2.set_title("Máscara Real (Ground Truth)")
                ax2.axis('off')
                ax3.imshow(preds_np[i].squeeze(), cmap='gray')
                ax3.set_title("Máscara Predicha por el Modelo")
                ax3.axis('off')
                ax4.imshow(img_display)
                ax4.imshow(preds_np[i].squeeze(), cmap='Reds', alpha=0.5)
                ax4.set_title("Predicción Superpuesta")
                ax4.axis('off')
                samples_shown += 1

print("Mostrando predicciones en el conjunto de validación...")
visualize_predictions(model, val_loader)